In [7]:
# Packages
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import sql

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import networkx as nx

from datetime import datetime, timedelta
from IPython.display import HTML

import os
from dotenv import load_dotenv
dotenv_path = os.path.expanduser('~/.pwds.env')
load_dotenv(dotenv_path)



# Conexão à base de dados
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')


conn = psycopg2.connect(
                host = db_host,
                port = db_port,
                user = db_user,
                password = db_password
        )

In [8]:
cur = conn.cursor()
cur.execute('''ROLLBACK;''')

In [9]:
def h(df):
    """
    Função que permite ver dataframe completa e de forma mais organizada
    """
    return HTML(df.to_html(index=False))

# Herfindahl-Hirschman Index

In [10]:
def contracts(n,cpv):
    """
    Função que retorna os IDs referentes a uma determinada categoria de CPV

    Argumentos:
    n (int): número de dígitos a considerar no slice do CPV ( 2: menor granularidade | 8: muito específico)
    cpv (str): primeiros n dígitos do cpv. Número de dígitos tem de coincidir com n, caso contrário retorna sempre 0 

    return: (tuple) IDs
    """
    
    cur = conn.cursor()

    query = '''
            SELECT id, data_publicacao, SUBSTRING(cpv,1,%s), nif1, nif2, adjudicataria
            FROM concursos_publicos
            WHERE SUBSTRING(cpv,1,%s) = %s;
            '''
    cur.execute(query,(n,n,cpv,))
    result = cur.fetchall()

    return result

In [11]:
gas = contracts(3,'652')
agua = contracts(3,'651')
energia = contracts(3,'653')
travel = contracts(3,'635')
internet = contracts(3,'724')
op_system = contracts(4,'4862')

In [12]:
# Cálculo do HHI para categoria de contratos referentes a comercialização de gás
gas_df_temp = pd.DataFrame(gas)
unique_nifs = gas_df_temp.iloc[:,4].unique()

hhi = list()

for i in unique_nifs:
    subdf = gas_df_temp[gas_df_temp.iloc[:,4] == i]
    hhi.append(len(subdf)/len(gas_df_temp))

soma = 0
for i in hhi:
    soma += (i*100)**2
print(soma)

1489.7837730609006


In [13]:
def hhi_calc(df):

    unique_nifs = df.iloc[:,4].unique()
    n = len(df)
    hhi = list()
    
    for i in unique_nifs:
        subdf = df[df.iloc[:,4] == i]
        hhi.append(len(subdf)/n)
    
    soma = 0
    for i in hhi:
        soma += (i*100)**2
        
    return len(unique_nifs), soma

In [14]:
# DF temporárias
gas_df_temp = pd.DataFrame(gas)
energia_df_temp = pd.DataFrame(energia)
agua_df_temp = pd.DataFrame(agua)
internet_df_temp = pd.DataFrame(internet)
travel_df_temp = pd.DataFrame(travel)
op_df_temp = pd.DataFrame(op_system)

In [15]:
for i in (gas_df_temp, energia_df_temp, agua_df_temp, internet_df_temp, travel_df_temp, op_df_temp):
    print(hhi_calc(i))

(13, 1489.7837730609006)
(40, 1051.2089999269474)
(31, 629.0174471992658)
(59, 824.7376244313307)
(69, 433.7826557976805)
(10, 1965.973534971644)


In [16]:
def cpv_x3():
    """
    Função que retorna categorias únicas de 3 dígitos do CPV

    return:
    result (dataframe): dataframe com duas colunas (categoria do contrato e número de contratos por categoria )
    """
    
    cur = conn.cursor()

    query = '''
            SELECT DISTINCT SUBSTRING(concursos_publicos."cpv", 1, 3) AS cpv3
            FROM concursos_publicos
            ORDER BY cpv3 DESC;
            '''
        
    cur.execute(query,)
    result = list(item[0] for item in cur.fetchall())
    
    return result

In [17]:
def contracts_v2(cpv):
    """
    Argumentos:
    cpv (str): primeiros 3 dígitos do cpv

    return: (tuple) IDs
    """
    
    cur = conn.cursor()

    query = '''
            SELECT id, data_publicacao, SUBSTRING(cpv,1,3), nif1, nif2, adjudicataria
            FROM concursos_publicos
            WHERE SUBSTRING(cpv,1,3) = %s;
            '''
    cur.execute(query,(cpv,))
    result = cur.fetchall()

    return result

In [18]:
cpv3 = cpv_x3()
n = len(cpv3)

In [23]:
hhi_df = pd.DataFrame(columns=('CPV','NrContratos','UniqueNIF', 'HHI'))
hhi_df

,CPV,NrContratos,UniqueNIF,HHI


In [25]:
for i,j in enumerate(cpv3):

    C = contracts_v2(j)
    df_temp = pd.DataFrame(C)
    n = len(df_temp)
    nnif, hhi = hhi_calc(df_temp)

    #print(f"CPV: {j} \t Número contratos: {n} \t\t Unique NIFs: {nnif} \t HHI: {round(hhi,1)} ")
    hhi_df.at[i,"CPV"] = j
    hhi_df.at[i,"NrContratos"] = n
    hhi_df.at[i,"UniqueNIF"] = nnif
    hhi_df.at[i,"HHI"] = round(hhi, 1)
        

In [26]:
h(hhi_df)

CPV,NrContratos,UniqueNIF,HHI
985,1,1,10000.0
983,662,289,72.2
982,2,2,5000.0
981,18,15,740.7
980,26,15,517.8
926,283,40,69.2
925,53,37,441.4
924,8,3,3750.0
923,347,162,96.7
922,19,11,1191.1


In [27]:
def hhi_limit(lb,ub):
    return hhi_df[(lb < hhi_df['HHI']) & (hhi_df['HHI'] <= ub)]

In [29]:
#hhi_limit(0,1500)      # Mercado não-concentrado
#hhi_limit(1500,2500)   # Mercado ligeiramente concentrado
hhi_limit(2500,10000)  # Mercado muito concentrado

,CPV,NrContratos,UniqueNIF,HHI
0,985,1,1,10000.0
2,982,2,2,5000.0
7,924,8,3,3750.0
39,778,1,1,10000.0
40,777,5,1,10000.0
41,775,4,1,10000.0
42,774,1,1,10000.0
47,766,1,1,10000.0
49,763,3,2,5555.6
53,753,1,1,10000.0


In [4]:
def cpv_x4():
    """
    Função que retorna categorias únicas de 3 dígitos do CPV

    return:
    result (dataframe): dataframe com duas colunas (categoria do contrato e número de contratos por categoria )
    """
    
    cur = conn.cursor()

    query = '''
            SELECT DISTINCT SUBSTRING(concursos_publicos."cpv", 1, 4) AS cpv4
            FROM concursos_publicos
            ORDER BY cpv4 DESC;
            '''
        
    cur.execute(query,)
    result = list(item[0] for item in cur.fetchall())
    
    return result

In [5]:
len(cpv_x4())

983